<a href="https://colab.research.google.com/github/williamshiaogit/DL-ML-AI-learning/blob/master/ptt_get.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime


def get_date_str(bias=0):
    """
    get the date as a string only shows month/day
    [bias] * as days want to add on today(could be minus)
           * defaut is zero stands for today
    """
    today = datetime.datetime.today()  # 獲得今天的日期
    date = (today + datetime.timedelta(days=bias)).strftime("%m/%d")  # 格式化日期
    return ' ' + date[1:] if date[0] == '0' else date     # 把0換成空白

# print(get_date_str())       # today
# print(get_date_str(-3))     # 3 days ago
# print(get_date_str(-7))     # 7 days ago
# print(get_date_str(100))    # day after 100 days


def Is_within_Target_Date_2020(this_date, target_date):
    # replace ' ' by '0' for the first letter
    this_date = '0' + this_date[1:] if this_date[0] == ' ' else this_date
    target_date = '0' + target_date[1:] if target_date[0] == ' ' else target_date

    this = datetime.date(2020, int(this_date[0:2]), int(this_date[3:]))
    target = datetime.date(2020, int(target_date[0:2]), int(target_date[3:]))

    # print for debugging
    # print("{} >= {} : {}" .format(this.strftime("%m/%d"), target.strftime("%m/%d"), this >= target))

    return this >= target

# Is_within_Target_Date_2020('03/12', '03/11')
# Is_within_Target_Date_2020('03/11', '03/11')
# Is_within_Target_Date_2020('03/10', '03/11')
# Is_within_Target_Date_2020(get_date_str(), get_date_str(-7))

In [ ]:
# -*- coding: utf-8 -*

import requests
from bs4 import BeautifulSoup as bs
#from day_computing import get_date_str, Is_within_Target_Date_2020


def ptt_listpage_crawler(url, day_bias=0):
    """
    crawl the ptt list page

    para::[url]
        - type: str
        - starting page URL
    para::[day_bias]
        - type: (minus) int
        - number of days before today
        - example: zero stands for today; -1 stands for yesterday 
    return::
        - [str] url of previous page; return None if achieved target date
        - [list] posts collection with target date
    """

    jar = requests.cookies.RequestsCookieJar()
    # 可把不同網頁的 cookie 設定進一個jar
    jar.set("over18", "1", domain="www.ptt.cc")
    # 將cookies加入request
    response = requests.get(url, cookies=jar).text
    # response為html格式，交由bs4解析
    html = bs(response)

    # 找到導航列
    navi_bar = html.find("div", class_="btn-group btn-group-paging")
    navi_bottons = navi_bar.find_all("a", class_="btn wide")
    # 從導航列取得上一頁的URL(next_url)
    for n in navi_bottons:
        if '上頁' in n.text:
            next_url = "https://www.ptt.cc" + n["href"]


    post_block = html.find("div", class_="r-list-container action-bar-margin bbs-screen")
    posts = post_block.find_all("div", class_="r-ent")
    post_list = []
    for post in posts:
        p = {}
        # Title - 文章標題
        try:
            p['title'] = post.find("div", class_="title").find("a").text
        except AttributeError:      # 如果文章已被刪除則略過
            p['title'] = post.find("div", class_="title").text
            if '刪除' in p['title']:
                continue
        if '公告' in p['title']:    # 如果是公告文則略過
            continue
        # URL - 文章連結
        try:
            url_path = post.find("div", class_="title").find("a")["href"]
            p['url'] = "https://www.ptt.cc" + url_path
            p['slug'] = '.'.join(url_path.split('/')[3].split('.')[:4])
            
        except:
            continue
        # DATE - 日期
        p['post_time'] = post.find("div", class_="date").text
        if not Is_within_Target_Date_2020(p['post_time'], get_date_str(day_bias)):
            next_url = None 
            continue  # 不在目標日期範圍內，略過此筆
        # PUSH - 推文數
        try:
            p['push'] = post.find("div", class_="nrec").find("span", class_="hl").text
        except AttributeError:      # 處理沒有人推文
            p['push'] = 0
        post_list.append(p)

    return next_url, post_list

# # Print for debugging
# for p in post_list:
#     for key, value in p.items():
#         print("{}: {}" .format(key, value))
#     print("------------------------------------------------")

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlretrieve
from zipfile import ZipFile
from os import remove
import ssl
import warnings


# Issue handling
ssl._create_default_https_context = ssl._create_unverified_context  # SSL
warnings.filterwarnings('ignore')  # 移除bs4的warning


def post_crawler(post_url, zipfile_name):
    jar = requests.cookies.RequestsCookieJar()
    # 可把不同網頁的 cookie 設定進一個jar
    jar.set("over18", "1", domain="www.ptt.cc")
    # 將cookies加入request
    response = requests.get(post_url, cookies=jar).text

    # response為html格式，交由bs4解析
    html = bs(response)

    main_content = html.find("div", class_="bbs-screen bbs-content")

    metas = main_content.find_all("span", class_="article-meta-tag")
    m_values = main_content.find_all("span", class_="article-meta-value")

    # Remove(extract) 作者 標題 時間 ------------------------
    meta = main_content.find_all("div", class_="article-metaline")
    for m in meta:
        m.extract()
    # Remove(extract) 看板名稱 ------------------------------
    right_meta = main_content.find_all("div", class_="article-metaline-right")
    for single_meta in right_meta:
        single_meta.extract()

    # Remove(extract) 推文前   ------------------------------
    datas = main_content.find_all("span", class_="f2")
    for data in datas:
        data.extract()

    # Remove(extract) 推文   --------------------------------
    pushes = main_content.find_all("div", class_="push")
    comments = []
    for single_push in pushes:
        push_tag = single_push.find("span", class_="push-tag").text.strip()
        push_userid = single_push.find("span", class_="push-userid").text.strip()
        push_content = single_push.find("span", class_="push-content").text.split(":")[1].strip()
        ipdatetime = single_push.find("span", class_="push-ipdatetime").text
        push_ip = ipdatetime.strip().split(" ")[0]
        push_time = " ".join(ipdatetime.strip().split(" ")[1:])
        comment = { "status": push_tag, "comment_id": push_userid, "content": push_content,"ip": push_ip,"comment_time": push_time }
        comments.append(comment)

    # Remove(extract) imgur圖片 ----------------------------
    # 1. 第一個部分，連結
    img_l = []
    photo_hrefs = main_content.find_all("a")
    for pic in photo_hrefs:
        if 'imgur' in pic["href"] and 'https' in pic["href"]:
            img_l.append(pic["href"])
            pic.extract()
    # # 2. 第二個部分，圖片顯示(richcontent)
    # richcontents = main_content.find_all("div", class_="richcontent")
    # for rich in richcontents:
    #   rich.extract()

    reply_msg = ""  # 開始搜集回應使用者的文字資訊

    # for (m, v) in zip(metas, m_values):
    #     print(m.text, ':', v.text)
    # for (m, v) in zip(metas, m_values):
    #     if m.text in ['標題', '時間']:
    #         reply_msg += m.text + ': ' + v.text + '\n'
    #     print(m.text, ':', v.text)
    # print(m_values[0].text)
    try:
        author = m_values[0].text        
    except:
        author = ''
    
    try:
        post_time = m_values[3].text
    except:
        post_time = ''

    # print("分數 :", score)
    # reply_msg += "分數: " + str(score) + '\n'
    # print("內文 :")
    # reply_msg += "內文: "

    # print(main_content.text)
    content = main_content.text
    # content_split = content.split('--')
    origin_content = content.split('--')[0]
    ori_l = origin_content.split("\n")
    ori_linted = []
    for o in ori_l:
        if o != "":
            ori_linted.append(o)
    # for i in ori_linted:
    #     reply_msg += (i + '\n')
    #     print(i)

    # print("圖片連結 :")
    img_name_list = []
    img_url_list = []
    for img in img_l:
        # 連結不含副檔名則加上.jpg
        if img[-4:] != ".gif" and img[-3:] != "jpg":
            img += ".jpg"
        # 下載圖片
        f = img.split('/')[-1]
        # urlretrieve(img, f)
        # print(img)
        img_name_list.append(f)
        img_url_list.append(img)

    return post_time, author, comments, img_name_list, img_url_list  # (回應使用者的文字資訊, 回傳圖片檔名list)


def img_dl_zip(img_url_list, zipfile_name):
    f_name = zipfile_name + '.zip'
    with ZipFile(f_name, 'w') as myzip:
        for img in img_url_list:
            # 連結不含副檔名則加上.jpg
            if img[-4:] != ".png" and img[-4:] != ".gif" and img[-3:] != "jpg":
                img += ".jpg"

            # 下載圖片、加入zip、刪除照片
            f = img.split('/')[-1]
            urlretrieve(img, f)
            print("downloaded: ", img)

            filename = img.split('/')[-1]
            myzip.write(filename)
            print("added to zip: ", filename)

            remove(filename)
            print("removed: ", filename)
    return f_name


def rm_img(img_name_list, num):
    '''
    param img_name_list: image file names
    param num: number of img to remove this time
                1, 2, 3, ...
                0 for remove all
    '''
    if num == 0:
        for img in img_name_list:
            print(img)
            remove(img)
    else:
        for _ in range(num):
            f = img_name_list.pop(0)
            remove(f)
            print(f)

In [ ]:
from time import sleep, time
#from listpage_crawler import ptt_listpage_crawler
#from postpage_crawler import post_crawler
import json

# 取得指定天數的 post list 資料，存成陣列
def get_post_list(day_bias = 7):
    # Input day_bias: 要查詢的天數(正整數)
    # return msg_r : 可傳給使用者看的查詢結果(string)
    # return df_r : 查詢結果組成的sorted DataFrame
    # PTT表特版首頁
    url = "https://www.ptt.cc/bbs/Beauty/index.html"

    day_bias = -day_bias + 1

    start_time = time()

    post_list = []
    delay_sec = 0.1
    count = 0
    if day_bias > 0:
        print("Check day_bias!!!")
    # while url != None:
    while url is not None:
        url, next_post_list = ptt_listpage_crawler(url, day_bias)
        print(url,'\n')
        post_list = post_list + next_post_list[::-1]
        count += 1
        sleep(delay_sec)

    end_time = time()
    duration = end_time - start_time
    print("花費時間: ",duration)
    return post_list







** 這裏可以設定爬的天數 N **

In [ ]:
n=1 #調整爬的天數

post_list = get_post_list(n)

print("共爬: ", n ," 天")

count=0
for index, post in enumerate(post_list):
    post_time, author, comments, img_name_list, img_url_list = post_crawler(post['url'], index)
    post_list[index]['post_time'] = post_time
    post_list[index]['imgs'] = img_url_list
    post_list[index]['author'] = author
    post_list[index]['comments'] = comments
    count+=1
    
    # 顯示每一頁的資料
    #print(post_list[index]['url'])
    #print(post_list[index],'\n')

print("\n==============================\n")
print("\n爬取po文資料 共",count, "筆\n")

# 直接暫存json 讀取
with open('data.json', 'w') as file:
    json.dump(post_list, file, ensure_ascii=False)

import json
with open('data.json') as json_file:
    jj = json.load(json_file)

count1=0
for p in jj:
    for img in p['imgs']:
        if img.split(".")[-1] == "jpg":
             try:
                count1 +=1
             except:
                pass
        else:
            continue
print("預計儲存所有圖片共：",count1,"張")

https://www.ptt.cc/bbs/Beauty/index3394.html 

None 

花費時間:  0.8218824863433838
共爬:  1  天



爬取po文資料 共 12 筆

預計儲存所有圖片共： 111 張
